In [1]:
import math
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from attack import (
    reconstruct_interactions,
    interaction_mia_fedrec,
)
from ranker import (
    LinearPDGDRanker,
    Neural1LayerPDGDRanker,
    Neural2LayerPDGDRanker,
    CollaborativeFilteringRecommender,
    NeuralCollaborativeFilteringRecommender,
)
from tqdm import tqdm
from utils import Metrics

In [27]:
# Simulation for LTR

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 10
num_features = 10
num_data = 100
lr = 1e-01
max_iter = 1000
num_atk = 1

metrics = Metrics()

models = {
    "linear_pdgd": LinearPDGDRanker(num_features),
    # "neural_1_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=5),
    # "neural_2_pdgd": Neural2LayerPDGDRanker(
    #     num_features, hidden_size=4, hidden_size2=2
    # ),
}

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features) * 2 - 1
    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))
    
    ranking = list(range(num_data))
    random.shuffle(ranking)
    ranking = torch.LongTensor(ranking)

    for model_name, model in models.items():
        params = model.gen_params()
        log_pos_bias_weight = model.calc_log_pos_bias_weight(
            ranking, model.forward_multiple(params, features), num_data
        )
        
        target = model.grad(
            params,
            features,
            ranking,
            interactions,
            log_pos_bias_weight=log_pos_bias_weight,
        )

        preds_raw = reconstruct_interactions(
            lambda I: model.grad(
                params, features, ranking, I, log_pos_bias_weight=log_pos_bias_weight
            ),
            target,
            num_data,
            lr=lr,
            max_iter=max_iter,
            num_rounds=num_atk,
            return_raw=True,
        )
        preds = preds_raw.sigmoid().round().long()

        metrics.update(model_name, interactions, preds, preds_raw=preds_raw)

    # Data manipulation
    if num_data > num_features:
        num_new_features = num_data - num_features
        new_features = torch.rand(num_data, num_new_features)
        features = torch.cat([features, new_features], dim=1)

        model = LinearPDGDRanker(num_features + num_new_features)
        params = model.gen_params()
        log_pos_bias_weight = model.calc_log_pos_bias_weight(
            ranking, model.forward_multiple(params, features), num_data
        )
        
        target = model.grad(
            params,
            features,
            ranking,
            interactions,
            log_pos_bias_weight=log_pos_bias_weight,
        )

        preds_raw = reconstruct_interactions(
            lambda I: model.grad(
                params, features, ranking, I, log_pos_bias_weight=log_pos_bias_weight
            ),
            target,
            num_data,
            lr=lr,
            max_iter=max_iter,
            num_rounds=num_atk,
            return_raw=True,
        )
        preds = preds_raw.sigmoid().round().long()

        metrics.update(model_name + "_DM", interactions, preds, preds_raw=preds_raw)

100%|██████████| 10/10 [05:11<00:00, 31.16s/it]


In [29]:
print(metrics.get_dataframe().to_string())

              name  accuracy        f1  precision    recall       auc    auc-pr extra_data
0      linear_pdgd    0.5425  0.548148   0.587302  0.513889  0.558399  0.607230         {}
1   linear_pdgd_DM    1.0000  1.000000   1.000000  1.000000  1.000000  1.000000         {}
2      linear_pdgd    0.4700  0.459184   0.505618  0.420561  0.500879  0.536752         {}
3   linear_pdgd_DM    1.0000  1.000000   1.000000  1.000000  1.000000  1.000000         {}
4      linear_pdgd    0.5250  0.520202   0.512438  0.528205  0.517724  0.485470         {}
5   linear_pdgd_DM    1.0000  1.000000   1.000000  1.000000  1.000000  1.000000         {}
6      linear_pdgd    0.4950  0.459893   0.457447  0.462366  0.504648  0.476432         {}
7   linear_pdgd_DM    1.0000  1.000000   1.000000  1.000000  1.000000  1.000000         {}
8      linear_pdgd    0.5200  0.505155   0.505155  0.505155  0.500651  0.472847         {}
9   linear_pdgd_DM    1.0000  1.000000   1.000000  1.000000  1.000000  1.000000         {}

In [11]:
# Simulation for collaborative filtering

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 10
num_features = 64
num_data = 1000
lr = 1e-01
max_iter = 100000
num_atk = 10

metrics = Metrics()

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features) * 2 - 1
    user_embedding = torch.rand(num_features) * 2 - 1
    user_embedding2 = torch.rand(num_features) * 2 - 1

    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))

    preds_raw = torch.rand((num_data),)
    metrics.update("Random", interactions, preds_raw.sigmoid().round().long(), preds_raw=preds_raw)

    # cf_rec = CollaborativeFilteringRecommender()
    # target = cf_rec.federated_item_grad(user_embedding, features, interactions)

    # preds_raw = reconstruct_interactions(
    #     lambda I: cf_rec.federated_item_grad(user_embedding2, features, I),
    #     target,
    #     num_data,
    #     lr=lr,
    #     max_iter=max_iter,
    #     num_rounds=num_atk,
    #     return_raw=True,
    # )
    # preds = preds_raw.sigmoid().round().long()

    # metrics.update("FCF_simple", interactions, preds, preds_raw=preds_raw)

    # preds_raw, user_embedding_est = reconstruct_interactions(
    #     lambda I, U: cf_rec.federated_item_grad(U, features, I),
    #     target,
    #     num_data,
    #     num_features,
    #     lr=lr,
    #     max_iter=max_iter,
    #     num_rounds=num_atk,
    #     return_raw=True,
    # )
    # preds = preds_raw.sigmoid().round().long()

    # embedding_err = F.mse_loss(user_embedding_est, user_embedding).item()

    # metrics.update(
    #     "FCF_private",
    #     interactions,
    #     preds,
    #     preds_raw=preds_raw,
    #     extra_data={"embedding_err": embedding_err},
    # )

    # preds = interaction_mia_fedrec(
    #     lambda I: cf_rec.federated_item_grad(user_embedding2, features, I),
    #     target,
    #     num_data,
    #     select_ratio=interactions.float().mean(),
    # )

    # metrics.update(
    #     "FCF_IMIA",
    #     interactions,
    #     preds,
    # )

    ncf_rec = NeuralCollaborativeFilteringRecommender(num_features, [4])

    # target = ncf_rec.item_grad(user_embedding, features, interactions.float())

    # preds_raw, user_embedding_est = reconstruct_interactions(
    #     lambda I, U: ncf_rec.item_grad(U, features, I),
    #     target,
    #     num_data,
    #     private_params_size=num_features,
    #     lr=lr,
    #     max_iter=max_iter,
    #     num_rounds=num_atk,
    #     return_raw=True,
    # )
    # preds = preds_raw.sigmoid().round().long()
    
    # embedding_err = F.mse_loss(user_embedding_est, user_embedding).item()

    # metrics.update(
    #     "FedNCF_private",
    #     interactions,
    #     preds,
    #     preds_raw=preds_raw,
    #     extra_data={"embedding_err": embedding_err},
    # )

    target = torch.cat(
        [
            ncf_rec.item_grad(user_embedding, features, interactions.float()).flatten(),
            ncf_rec.feature_grad(user_embedding, features, interactions.float()),
        ]
    )

    preds_raw, user_embedding_est = reconstruct_interactions(
        lambda I, U: torch.cat(
            [
                ncf_rec.item_grad(U, features, I).flatten(),
                ncf_rec.feature_grad(U, features, I, retain_graph=True),
            ]
        ),
        target,
        num_data,
        private_params_size=num_features,
        lr=lr,
        max_iter=max_iter,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()

    embedding_err = F.mse_loss(user_embedding_est, user_embedding).item()

    metrics.update(
        "FedNCF_private2",
        interactions,
        preds,
        preds_raw=preds_raw,
        extra_data={"embedding_err": embedding_err},
    )

    target = ncf_rec.item_grad(user_embedding, features, interactions.float())

    preds = interaction_mia_fedrec(
        lambda I: ncf_rec.item_grad(user_embedding2, features, I.float()),
        target,
        num_data,
        select_ratio=interactions.float().mean(),
    )

    metrics.update(
        "FedNCF_IMIA",
        interactions,
        preds,
    )

100%|██████████| 10/10 [00:18<00:00,  1.82s/it]


In [12]:
print(metrics.get_dataframe().to_string())

               name  accuracy        f1  precision    recall       auc    auc-pr                             extra_data
0            Random     0.486  0.654105   0.486000  1.000000  0.514337  0.491345                                     {}
1   FedNCF_private2     0.501  0.499498   0.487280  0.512346  0.502710  0.491472   {"embedding_err": 0.806613028049469}
2       FedNCF_IMIA     0.530  0.516461   0.516461  0.516461       NaN       NaN                                     {}
3            Random     0.530  0.692810   0.530000  1.000000  0.493497  0.538038                                     {}
4   FedNCF_private2     0.491  0.509161   0.520710  0.498113  0.494886  0.522588  {"embedding_err": 0.7423382997512817}
5       FedNCF_IMIA     0.522  0.549057   0.549057  0.549057       NaN       NaN                                     {}
6            Random     0.524  0.687664   0.524000  1.000000  0.500012  0.519754                                     {}
7   FedNCF_private2     0.492  0.499014 